In [3]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import sys
import os
import pandas as pd

Search for "zeropoint" in the image header

In [15]:
def sum_pixel_values_within_radius(image_file, center, radius):
    """
    Sum the pixel values within a certain radius in a FITS file.
    
    Parameters:
    - image_file (str): The name of the FITS file.
    - center (tuple): The (x, y) pixel coordinates for the center of the circle.
    - radius (float): The radius of the circle.
    
    Returns:
    - float: The sum of the pixel values within the specified radius.
    """
    # Open the FITS file
    with fits.open(image_file) as hdul:
        # Find the first extension with image data
        for hdu in hdul:
            if hdu.data is not None:
                image_data = hdu.data
                break
        else:
            raise ValueError("No image data found in the FITS file.")
    
    # Get the dimensions of the image
    height, width = image_data.shape
    
    # Create a grid of coordinates
    y, x = np.ogrid[:height, :width]
    
    # Calculate the distance of each pixel from the center
    distance = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    
    # Create a mask for pixels within the specified radius
    mask = distance <= radius
    
    # Sum the pixel values within the mask
    pixel_sum = np.sum(image_data[mask])
    
    return pixel_sum

In [18]:
# Example usage
image_file = r'C:\\Users\\admin1\\OneDrive\Desktop\\134L Images\\good_fits\\van_Maanen_g.fits'
center = (2000, 705)
radius = 5
pixel_sum = sum_pixel_values_within_radius(image_file, center, radius)
print(f'The sum of pixel values within the radius is {pixel_sum}')

The sum of pixel values within the radius is 3352.44775390625


Calculate temperature using color index data

In [17]:
def load_color_index_data(file_path):
    return pd.read_csv(file_path)

In [32]:
data = load_color_index_data('LCO_bb_color_indices.csv')

In [18]:
def calculate_temperature(color_index, color_index_type, data):
    """
    Calculate the temperature for a given color index and color index type using the data from the CSV file.
    """
    # Ensure the color index type is valid
    if color_index_type not in data.columns:
        raise ValueError(f"Invalid color index type: {color_index_type}. Must be one of {data.columns[1:]}")
    
    # Find the closest color index in the data
    closest_row = data.iloc[(data[color_index_type] - color_index).abs().argsort()[:1]]
    return closest_row['Temperature (K)'].values[0]


In [39]:
# Example usage
color_index = 0.65
color_index_type = 'g-r'
temperature = calculate_temperature(color_index, color_index_type, data)
print(f'The temperature for color index {color_index} with color index type {color_index_type} is {temperature} K')

The temperature for color index 0.65 with color index type g-r is 5400 K


Bring in functions from the other doc

In [7]:
# coordinates in the form [x, y]
van_maanen_coords = (1189, 1173)
lawd8_coords = (1208, 1216)

In [ ]:
# open a fits file
# file = r'C:\Users\admin1\OneDrive\Desktop\134L Images\good_fits\van_Maanen_g.fits' # desktop
file = r'C:\Users\Willi\Desktop\134L Images\good_fits\van_Maanen_g.fits' # laptop
os.path.exists(file)
hdul = fits.open(file)
data = hdul[1].data
print(data.shape)

(2400, 2400)


In [12]:
def apparent_mag(flux, zeropoint):
    return -2.5 * np.log10(flux) + zeropoint

In [6]:
filters = ['g', 'r', 'i']
filter_zeropoints = [21.65063765247328, 21.025287319733366, 20.03855919309204] # instrumental zeropoint (mag), for g, r, i
color_index_types = ['g-r', 'r-i', 'g-i']

In [28]:
# for van Maanen's star
star = 'van_Maanen_'
radius = 8 # pixels
VM_fluxes = []
x_star, y_star = 1189, 1173

file_root = r'C:\Users\Willi\Desktop\134L Images\good_fits'
for filter in filters:
    file = os.path.join(file_root, star + f"{filter}" + '.fits')
    # print(file)

    hdul = fits.open(file)
    data = hdul[1].data

    y, x = np.indices(data.shape)  # Create coordinate grids for the image
    distance = np.sqrt((x - x_star)**2 + (y - y_star)**2)  # Distance of each pixel from the star center

    aperture_mask = distance <= radius

    background_mask = (distance > radius + 1) & (distance <= radius + 10)
    background_values = data[background_mask]  # Pixels outside the aperture
    background_median = np.median(background_values)  # Median background level

    aperture_flux = np.sum(data[aperture_mask] - background_median)

    VM_fluxes.append(aperture_flux)

print(VM_fluxes)

[170625.5, 123303.49, 51127.43]


In [10]:
def calculate_flux(file_root, star_name, x_position, y_position, radius):
    fluxes = []
    for filter in filters:
        file = os.path.join(file_root, star_name + f"{filter}" + '.fits')

        hdul = fits.open(file)
        data = hdul[1].data

        y, x = np.indices(data.shape)  # Create coordinate grids for the image
        distance = np.sqrt((x - x_position)**2 + (y - y_position)**2)  # Distance of each pixel from the star center

        aperture_mask = distance <= radius

        background_mask = (distance > radius + 1) & (distance <= radius + 10)
        background_values = data[background_mask]  # Pixels outside the aperture
        background_median = np.median(background_values)  # Median background level

        aperture_flux = np.sum(data[aperture_mask] - background_median)

        fluxes.append(aperture_flux)

    return fluxes

In [11]:
calculate_flux(r'C:\Users\Willi\Desktop\134L Images\good_fits', 'van_Maanen_', 1189, 1173, 8)

[170625.5, 123303.49, 51127.43]

In [47]:
VM_index_g = mag(VM_fluxes[0], filter_zeropoints[0])
VM_index_r = mag(VM_fluxes[1], filter_zeropoints[1])
VM_index_i = mag(VM_fluxes[2], filter_zeropoints[2])
print(VM_index_g, VM_index_r, VM_index_i)

VM_gr = VM_index_g - VM_index_r
VM_ri = VM_index_r - VM_index_i
VM_gi = VM_index_g - VM_index_i
print(VM_gr, VM_ri, VM_gi)

VM_indices = [VM_gr, VM_ri, VM_gi]

8.570528294639416 8.297849108292326 8.266924375968628
0.27267918634709076 0.030924732323697413 0.3036039186707882


In [33]:
color_index_type = 'g-r'
temperature = calculate_temperature(VM_gr, color_index_type, data)
print(f"The temperature for van Maanen's star color index {color_index} with color index type {color_index_type} is {temperature} K")

The temperature for van Maanen's star color index 0.65 with color index type g-r is 7600 K


In [22]:
color_index_type = 'r-i'
temperature = calculate_temperature(VM_ri, color_index_type, data)
print(f"The temperature for van Maanen's star color index {color_index} with color index type {color_index_type} is {temperature} K")

The temperature for van Maanen's star color index 0.65 with color index type r-i is 10000 K


In [23]:
color_index_type = 'g-i'
temperature = calculate_temperature(VM_gi, color_index_type, data)
print(f"The temperature for van Maanen's star color index {color_index} with color index type {color_index_type} is {temperature} K")

The temperature for van Maanen's star color index 0.65 with color index type g-i is 8200 K


In [24]:
print('Average temp for van Maanen: ' , (7600+10000+8200)/3)

Average temp for van Maanen:  8600.0


In [48]:
temperatures = []
for i in range(3):
    temperature = calculate_temperature(VM_indices[i], color_index_types[i], data)
    temperatures.append(temperature)
    # print(f"The temperature for LAWD 8 color index {color_index} with color index type {color_index_type} is {temperature} K")

print('Average temp for van Maanen: ' , np.mean(temperatures))

Average temp for van Maanen:  8600.0


Now for LAWD 8

In [27]:
# for LAWD 8
star = 'LAWD_8_'
radius = 7 # pixels
lawd8_fluxes = []
x_star, y_star = 1208, 1216

file_root = r'C:\Users\Willi\Desktop\134L Images\good_fits'
for filter in filters:
    file = os.path.join(file_root, star + f"{filter}" + '.fits')
    # print(file)

    hdul = fits.open(file)
    data = hdul[1].data

    y, x = np.indices(data.shape)  # Create coordinate grids for the image
    distance = np.sqrt((x - x_star)**2 + (y - y_star)**2)  # Distance of each pixel from the star center

    aperture_mask = distance <= radius

    background_mask = (distance > radius + 1) & (distance <= radius + 10)
    background_values = data[background_mask]  # Pixels outside the aperture
    background_median = np.median(background_values)  # Median background level

    aperture_flux = np.sum(data[aperture_mask] - background_median)

    lawd8_fluxes.append(aperture_flux)

print(lawd8_fluxes)

[67049.05, 38889.344, 11649.363]


In [45]:
lawd8_index_g = mag(lawd8_fluxes[0], filter_zeropoints[0])
lawd8_index_r = mag(lawd8_fluxes[1], filter_zeropoints[1])
lawd8_index_i = mag(lawd8_fluxes[2], filter_zeropoints[2])
print(lawd8_index_g, lawd8_index_r, lawd8_index_i)

lawd8_gr = lawd8_index_g - lawd8_index_r
lawd8_ri = lawd8_index_r - lawd8_index_i
lawd8_gi = lawd8_index_g - lawd8_index_i
print(lawd8_gr, lawd8_ri, lawd8_gi)
lawd8_indices = [lawd8_gr, lawd8_ri, lawd8_gi]

9.584656025962659 9.55071132333444 9.872803444343262
0.03394470262821869 -0.3220921210088221 -0.2881474183806034


In [46]:
temperatures = []
for i in range(3):
    temperature = calculate_temperature(lawd8_indices[i], color_index_types[i], data)
    temperatures.append(temperature)
    # print(f"The temperature for LAWD 8 color index {color_index} with color index type {color_index_type} is {temperature} K")

print('Average temp for LAWD 8: ' , np.mean(temperatures))

Average temp for LAWD 8:  11400.0
